In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import linalg
from matplotlib import colors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix  

%matplotlib inline

In [14]:
df = pd.read_csv('../datasets/01/all.csv', sep='\t')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5555 entries, 0 to 5554
Data columns (total 54 columns):
D feed flow (stream 2)                                        5555 non-null float64
E feed flow (stream 3)                                        5555 non-null float64
A feed flow (stream 1)                                        5555 non-null float64
A and C feed flow (stream 4)                                  5555 non-null float64
Compressor recycle valve                                      5555 non-null float64
Purge valve (stream 9)                                        5555 non-null float64
Separator pot liquid flow (stream 10)                         5555 non-null float64
Stripper liquid product flow (stream 11)                      5555 non-null float64
Stripper steam valve                                          5555 non-null float64
Reactor cooling water flow                                    5555 non-null float64
Condenser cooling water flow                         

In [16]:
df.head()

,D feed flow (stream 2),E feed flow (stream 3),A feed flow (stream 1),A and C feed flow (stream 4),Compressor recycle valve,Purge valve (stream 9),Separator pot liquid flow (stream 10),Stripper liquid product flow (stream 11),Stripper steam valve,Reactor cooling water flow,...,Purge gas analysis - Component E,Purge gas analysis - Component F,Purge gas analysis - Component G,Purge gas analysis - Component H,Product analysis - Component D,Product analysis - Component E,Product analysis - Component F,Product analysis - Component G,Product analysis - Component H,class
0,0.24692,3610.0,4443.3,9.8489,26.853,43.254,2662.9,71.272,120.32,0.33455,...,66.651,14.094,40.377,35.648,35.903,47.504,39.202,33.274,50.0,C00000000000000000000
1,0.24692,3610.0,4443.3,10.1300,27.000,43.237,2655.3,69.850,120.27,0.33681,...,68.515,15.503,40.102,44.218,45.663,46.742,37.082,22.488,50.0,C00000000000000000000
2,0.24696,3610.0,4443.3,10.3810,26.981,42.996,2695.0,68.720,120.43,0.33723,...,70.124,18.156,40.049,45.793,52.033,46.415,40.665,15.057,50.0,C00000000000000000000
3,0.24696,3610.0,4443.3,10.5380,26.936,43.020,2752.3,68.195,120.52,0.33694,...,71.115,19.719,40.084,43.264,53.568,46.698,42.488,12.792,50.0,C00000000000000000000
4,0.39983,3771.2,4602.3,10.5760,26.891,43.468,2806.9,68.384,120.70,0.26767,...,71.320,19.709,32.032,39.782,51.799,47.278,47.920,14.191,50.0,C00000000000000000000


In [17]:
df.describe()

,D feed flow (stream 2),E feed flow (stream 3),A feed flow (stream 1),A and C feed flow (stream 4),Compressor recycle valve,Purge valve (stream 9),Separator pot liquid flow (stream 10),Stripper liquid product flow (stream 11),Stripper steam valve,Reactor cooling water flow,...,Purge gas analysis - Component D,Purge gas analysis - Component E,Purge gas analysis - Component F,Purge gas analysis - Component G,Purge gas analysis - Component H,Product analysis - Component D,Product analysis - Component E,Product analysis - Component F,Product analysis - Component G,Product analysis - Component H
count,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,...,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.000000,5555.0
mean,0.286005,3930.555878,4232.527561,9.339063,26.901984,42.890898,2615.317642,74.997560,120.400061,0.322940,...,28.837306,63.212109,12.627860,40.891921,39.836570,48.284986,34.927968,43.741242,17.043278,50.0
std,0.051279,19.630619,72.640202,0.159948,0.004230,0.111870,20.359386,0.728325,0.009477,0.012242,...,5.070047,1.048899,1.509403,1.707247,0.250072,0.241251,8.451609,0.611452,0.325140,0.0
min,0.001758,3610.000000,3798.600000,7.674000,26.853000,41.916000,2536.700000,68.195000,120.270000,0.165050,...,0.001044,52.294000,1.493100,19.058000,35.648000,35.903000,21.011000,37.082000,12.792000,50.0
25%,0.284360,3932.700000,4228.600000,9.342100,26.902000,42.889000,2613.800000,75.000000,120.400000,0.323220,...,28.680000,63.232000,12.565000,40.948000,39.822000,48.293000,33.515000,43.738000,17.013000,50.0
50%,0.284360,3932.700000,4228.600000,9.342100,26.902000,42.889000,2613.800000,75.000000,120.400000,0.323220,...,28.680000,63.232000,12.565000,40.948000,39.822000,48.293000,33.515000,43.738000,17.013000,50.0
75%,0.284360,3932.700000,4228.600000,9.342100,26.902000,42.889000,2613.800000,75.000000,120.400000,0.323220,...,28.680000,63.232000,12.565000,40.948000,39.822000,48.293000,33.515000,43.738000,17.013000,50.0
max,0.856070,4002.300000,5016.800000,10.576000,27.000000,44.165000,2888.200000,81.657000,120.700000,0.384740,...,84.919000,71.320000,25.661000,50.016000,45.793000,53.568000,100.100000,49.144000,33.274000,50.0


In [18]:
X = df.iloc[:,0:53]
y = df['class']
n_neighbors = 20

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)  

In [22]:
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
clf = qda.fit(X, y)

/home/jvrmaia/.pyenv/versions/3.7.3/envs/pr-trab01/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [23]:
y_pred = clf.predict(X_test)

In [24]:
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))  

[[ 40   0]
 [  2 514]]
                       precision    recall  f1-score   support

C00000000000000000000       0.95      1.00      0.98        40
C00000000100000000000       1.00      1.00      1.00       516

            micro avg       1.00      1.00      1.00       556
            macro avg       0.98      1.00      0.99       556
         weighted avg       1.00      1.00      1.00       556

